In [ ]:
 # to conncet the gdrive and the google colab
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#to change the current 
%cd /content/gdrive/MyDrive/DS636/midterm_project/data/
!pwd

/content/gdrive/MyDrive/DS636/midterm_project/data
/content/gdrive/MyDrive/DS636/midterm_project/data


In [ ]:
import pandas as pd
import os 
from numpy import floor
from statistics import mean
from decimal import *
import numpy as np

# pd.set_option('display.max_rows', 5)
# getcontext().prec = 5

In [ ]:
ibm_df = pd.read_csv("IBM.csv", usecols=[0, 4, 5])
msft_df = pd.read_csv("MSFT.csv", usecols=[0, 4, 5])
goog_df = pd.read_csv("GOOG.csv", usecols=[0, 4, 5])
amzn_df = pd.read_csv("AMZN.csv", usecols=[0, 4, 5])
meta_df = pd.read_csv("META.csv", usecols=[0, 4, 5])
nflx_df = pd.read_csv("NFLX.csv", usecols=[0, 4, 5])
tsla_df = pd.read_csv("TSLA.csv", usecols=[0, 4, 5])
orcl_df = pd.read_csv("ORCL.csv", usecols=[0, 4, 5])
sap_df = pd.read_csv("SAP.csv", usecols=[0, 4, 5])
aapl_df = pd.read_csv("AAPL.csv", usecols=[0, 4, 5])

foreignExchange_df = pd.read_csv("JPY=X.csv")


In [ ]:
# Create a list of stocks
stock_container = [ibm_df, msft_df, goog_df, aapl_df, amzn_df]
universal_container = [ibm_df, msft_df, goog_df, aapl_df, amzn_df, meta_df, nflx_df, tsla_df, orcl_df, sap_df]
  
# Set names for the lists
stock_names = ["IBM", "MSFT", "GOOG", "AAPL", "AMZN"]
universal_names = ["IBM", "MSFT", "GOOG", "AAPL", "AMZN", "FB", "NFLX", "TSLA", "ORCL", "SAP"]
stock_container = dict(zip(stock_names, stock_container))
universal_container = dict(zip(universal_names, universal_container))

# For display purposes
print(stock_container.keys())
print(universal_container.keys())

dict_keys(['IBM', 'MSFT', 'GOOG', 'AAPL', 'AMZN'])
dict_keys(['IBM', 'MSFT', 'GOOG', 'AAPL', 'AMZN', 'FB', 'NFLX', 'TSLA', 'ORCL', 'SAP'])


In [ ]:
def compute_stock_price(df,day=0,wallet=1000000):
    df = df.reset_index(drop=True)
    close_price = df.loc[day, 'Close']
    stock_units = wallet / close_price
    return int(stock_units) * close_price

In [ ]:
def compute_cash_balance(df, day=0, wallet=1000000):
    # df = pd.DataFrame(df)
    close_price = df.loc[day,"Close"]
    stock_units = wallet / close_price
    remainder = stock_units - np.floor(stock_units)
    cash_balance = remainder * close_price
    
    return cash_balance

In [ ]:
def compute_stock_units(df, day=0, wallet=1000000):
    df = pd.DataFrame(df)
    close_price = df["Close"].iloc[day,]
    
    stock_units = wallet / close_price
    return floor(stock_units)

In [ ]:
def compute_mtm(portfolio, day=0):
    cash_balance = portfolio.iloc[day, 1]
    share_price = portfolio.iloc[day, 2]
    
    return cash_balance + share_price

In [ ]:
def compute_percent(df):
  '''
  This function takes a df computes the average and stores on the 5 day and a df with two extra columns 
  '''
  adjclose=df['Adj Close']
  lag_adjclose=df['Adj Close'].shift(1)
  difference=(adjclose/lag_adjclose-1)*100
  average_difference=[0]*len(adjclose)
  for index in range(5,len(adjclose),5):
    average=float(difference[index-5:index].mean(skipna=True))
    average_difference[index]=average
  df['percent_difference']=difference
  df['Average_precent_difference']=average_difference
  return df

In [ ]:
def compute_dividend(df):
  date=df['Date']
  date=date[1:len(date)]

  close=df["Close"]
  adjclose=df['Adj Close']

  lag_close=df['Close'].shift(1)
  lag_adjclose=df['Adj Close'].shift(1)


  dividend=round(((lag_close/close)-(lag_adjclose/adjclose))*close,2)
  temp_df=temp_df=pd.DataFrame(
      {
        "Date": date,
        "Dividend" :dividend
      }
  )
  dividend_frame = temp_df[temp_df["Dividend"] != 0]

  df=pd.merge(df,dividend_frame,how="left",on="Date")
  return df

In [ ]:
def compute_dividend_sum(adjusted_close_ref, stock_unit_list, day=0):
    total_dividend_sum = 0
    
    for stock in adjusted_close_ref:
        dividend_amount = stock.iloc[day].get('Dividends')
        
        if not pd.isna(dividend_amount):
            unit_amount = stock_unit_list[i]
            total_dividend_sum += unit_amount * dividend_amount
    
    return total_dividend_sum

In [ ]:
def compute_portfolio(stock_container,portfolio,day=1,rebalance_flag=1, adjusted_close_ref=None, stock_unit_list=None, descending=False):
    wallet = portfolio.iloc[day-1, 4]
    share_price = sum([compute_share_price(stock_container[df], day, wallet) for df in stock_container])
    dividend_days_sums = compute_dividend_sum(adjusted_close_ref, stock_unit_list, day)
    cash_accumlation = portfolio.iloc[day-1, 1]
    
    portfolio.iloc[day, 1] = cash_accumlation + dividend_days_sums
    portfolio.iloc[day, 2] = share_price
    portfolio.iloc[day, 4] = wallet

    mtm = compute_mtm(portfolio, day)
    
    portfolio.iloc[day, 3] = mtm
    
    if rebalance_flag % 5 != 0:
        return [portfolio, None]
    else:
        # perform rebalancing operations
        portfolio.iloc[iter, 4] = portfolio.iloc[iter, 3] / 5  # dispersion
        share_price = sum([compute_share_price(df, day, portfolio.iloc[iter, 4]) for df in stock_container])
        cash_accumlation = sum([compute_cash_balance(df, day, portfolio.iloc[iter, 4]) for df in stock_container])
        
        portfolio.iloc[day, 1] = cash_accumlation
        portfolio.iloc[day, 2] = share_price        
        
        adjusted_reference_guide = [adjusted_close_ref[name][day][4] for name in adjusted_close_ref.keys()]
        ordered_frame = pd.DataFrame({"Stock": list(adjusted_close_ref.keys()), "Avg": adjusted_reference_guide})
        
        # Use strategy 1 or strategy 2
        if not descending:
            ordered_frame = ordered_frame.sort_values("Avg")
        else:
            ordered_frame = ordered_frame.sort_values("Avg", ascending=False)
        
        return [portfolio, list(ordered_frame.iloc[:5, 0])]


In [ ]:
universe_len = len(universal_container["GOOG"])

dates = universal_container['GOOG']
dates = dates["Date"]

stock_unit_list = [compute_stock_units(universal_container[container]) for container in universal_container]

portfolio_strat1 = pd.DataFrame({"Date": dates, 
                       "Cash_Balance": [None] * universe_len,
                       "Sum_Share_Prices": [None] * universe_len,
                       "MTM": [None] * universe_len,
                      "Wallet_Per_Stock": [None] * universe_len})

adjusted_close_ref = [compute_percent(universal_container[container]) for container in universal_container]
adjusted_close_ref = list(map(compute_dividend, adjusted_close_ref))

# percent_mutated = list(map(compute_percent, universal_container))
# adjusted_close_ref = list(map(compute_dividend, percent_mutated))



# adjusted_close_ref = [compute_dividend(compute_percent(universal_container[stock]) for stock in universal_container]



portfolio_strat1.iloc[0,1] = sum([compute_cash_balance(stock_container[stock]) for stock in stock_container])
portfolio_strat1.iloc[0,2] = sum([compute_share_price(stock_container[stock]) for stock in stock_container])
portfolio_strat1.iloc[0,3] = compute_mtm(portfolio_strat1)

portfolio_strat1.iloc[0,4] = 1000000
portfolio_strat1.iloc[1,4] = 1000000

rebalance_flag = 1
for i in range(1, universe_len):
    
    # Receive vector of calculations    
    calculations = compute_portfolio(stock_container, portfolio_strat1, day=i,
                                  rebalance_flag=rebalance_flag, adjusted_close_ref=adjusted_close_ref, 
                                  stock_unit_list=stock_unit_list)
    
    portfolio_strat1 = calculations[0]
    newStocks = calculations[1]
        
    if rebalanceFlag % 5 != 0:
        rebalanceFlag =+ 1
    else:
        
        # update stocks to buy
        rebalanceFlag = 1
        stockContainer = universal_container[newStocks]
        stockContainer = dict(zip(newStocks, stockContainer.values()))


jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
j

In [ ]:
stock_container = [ibm_df, msft_df, goog_df, aapl_df, amzn_df]
names = ["IBM", "MSFT", "GOOG", "AAPL", "AMZN"]
universe_len = len(universal_container["GOOG"])
stock_container = dict(zip(names, stock_container))

dates = universal_container['GOOG']
dates = dates["Date"]
stock_unit_list = [compute_stock_units(universal_container[container]) for container in universal_container]

portfolio_strat2 = pd.DataFrame({"Date": dates, 
                       "Cash_Balance": [None] * universe_len,
                       "Sum_Share_Prices": [None] * universe_len,
                       "MTM": [None] * universe_len,
                      "Wallet_Per_Stock": [None] * universe_len})

adjusted_close_ref = [compute_percent(universal_container[container]) for container in universal_container]
adjusted_close_ref = list(map(compute_dividend, adjusted_close_ref))

# percent_mutated = list(map(compute_percent, universal_container))
# adjusted_close_ref = list(map(compute_dividend, percent_mutated))



# adjusted_close_ref = [compute_dividend(compute_percent(universal_container[stock]) for stock in universal_container]



portfolio_strat2.iloc[0,1] = sum([compute_cash_balance(stock_container[stock]) for stock in stock_container])
portfolio_strat2.iloc[0,2] = sum([compute_share_price(stock_container[stock]) for stock in stock_container])
portfolio_strat2.iloc[0,3] = compute_mtm(portfolio_strat1)

portfolio_strat2.iloc[0,4] = 1000000
portfolio_strat2.iloc[1,4] = 1000000

rebalance_flag = 1
for i in range(1, universe_len-2):
    
    # Receive vector of calculations    
    calculations = compute_portfolio(stock_container, portfolio_strat2, day=i,
                                  rebalance_flag=rebalance_flag, adjusted_close_ref=adjusted_close_ref, 
                                  stock_unit_list=stock_unit_list,descending=True)
    
    portfolio_strat1 = calculations[0]
    newStocks = calculations[1]
        
    if rebalanceFlag % 5 != 0:
        rebalanceFlag =+ 1
    else:
        
        # update stocks to buy
        rebalanceFlag = 1
        stockContainer = universal_container[newStocks]
        stockContainer = dict(zip(newStocks, stockContainer.values()))


jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
jj
j

In [ ]:

portfolio_strat1['Date'] = pd.to_datetime(portfolio_strat2['Date'])
 

In [ ]:
portfolio_strat1.head(15)

,Date,Cash_Balance,Sum_Share_Prices,MTM,Wallet_Per_Stock
0,2018-01-02,1269.665796,4998730.334204,5000000.0,1000000
1,2018-01-03,1269.665796,4998505.202299,4999774.868095,1000000
2,2018-01-04,1269.665796,4998406.023127,4999675.688923,1000000
3,2018-01-05,1269.665796,4998934.095933,5000203.761729,1000000
4,2018-01-08,1269.665796,4999359.492246,5000629.158042,1000000
5,2018-01-09,1269.665796,4998489.554439,4999759.220235,1000000
6,2018-01-10,1269.665796,4998525.49238,4999795.158176,1000000
7,2018-01-11,1269.665796,4998972.470205,5000242.136001,1000000
8,2018-01-12,1269.665796,4999558.226791,5000827.892587,1000000
9,2018-01-16,1269.665796,4998913.495094,5000183.16089,1000000


In [ ]:
portfolio_strat2.head(15)

,Date,Cash_Balance,Sum_Share_Prices,MTM,Wallet_Per_Stock
0,2018-01-02,1269.665796,4998730.334204,5000000.0,1000000
1,2018-01-03,1269.665796,4998505.202299,4999774.868095,1000000
2,2018-01-04,1269.665796,4998406.023127,4999675.688923,1000000
3,2018-01-05,1269.665796,4998934.095933,5000203.761729,1000000
4,2018-01-08,1269.665796,4999359.492246,5000629.158042,1000000
5,2018-01-09,1269.665796,4998489.554439,4999759.220235,1000000
6,2018-01-10,1269.665796,4998525.49238,4999795.158176,1000000
7,2018-01-11,1269.665796,4998972.470205,5000242.136001,1000000
8,2018-01-12,1269.665796,4999558.226791,5000827.892587,1000000
9,2018-01-16,1269.665796,4998913.495094,5000183.16089,1000000


In [ ]:
portfolio_strat2.sort_values(by='Date')


,Date,Cash_Balance,Sum_Share_Prices,MTM,Wallet_Per_Stock
0,2018-01-02,1269.665796,4998730.334204,5000000.0,1000000
1,2018-01-03,1269.665796,4998505.202299,4999774.868095,1000000
2,2018-01-04,1269.665796,4998406.023127,4999675.688923,1000000
3,2018-01-05,1269.665796,4998934.095933,5000203.761729,1000000
4,2018-01-08,1269.665796,4999359.492246,5000629.158042,1000000
...,...,...,...,...,...
245,2018-12-21,1269.665796,4997648.202602,4998917.868398,1000000
246,2018-12-24,1269.665796,4999444.500848,5000714.166644,1000000
247,2018-12-26,1269.665796,4998662.537377,4999932.203173,1000000
248,2018-12-27,None,None,None,None


In [ ]:
close_values = pd.DataFrame([universal_container[stock["Close"]] for stock in universal_container])

# Compute mean and add as new column "HTI"
hti_frame = close_values.assign(HTI=close_values.mean(axis=1))

# Compute percentage change relative to first value
hti_frame = hti_frame.assign(
    HTI_PERCENT_CHANGE=((hti_frame["HTI"] - hti_frame["HTI"][0]) / hti_frame["HTI"][0]) * 100
)
hti_frame["HTI_PERCENT_CHANGE"] = round(hti_frame["HTI_PERCENT_CHANGE"], 3)

# Print first few rows of resulting DataFrame
print(hti_frame.head())

TypeError: ignored

In [ ]:
for stock in universal_container:
  print(universal_container[stock]['Close'])

0      131.443588
1      132.466537
2      131.424469
3      133.059280
4      132.820267
          ...    
247    127.730003
248    126.300003
249    124.930000
250    125.309998
251    122.709999
Name: Close, Length: 252, dtype: float64
0       85.949997
1       86.349998
2       87.110001
3       88.190002
4       88.279999
          ...    
245     98.230003
246     94.129997
247    100.559998
248    101.180000
249    100.389999
Name: Close, Length: 250, dtype: float64
0      1065.000000
1      1082.479980
2      1086.400024
3      1102.229980
4      1106.939941
          ...     
245     979.539978
246     976.219971
247    1039.459961
248    1043.880005
249    1037.079956
Name: Close, Length: 250, dtype: float64
0      43.064999
1      43.057499
2      43.257500
3      43.750000
4      43.587502
         ...    
245    37.682499
246    36.707500
247    39.292500
248    39.037498
249    39.057499
Name: Close, Length: 250, dtype: float64
0      1189.010010
1      1204.199951
2     